### Data Ingestion
Export Data from MongoDB and read it in jupyter

In [11]:
import pandas as pd
df = pd.read_json("sales.json", lines=True)
df.head()

,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod
0,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online
1,{'$oid': '5bd761dcae323e45a93ccfe9'},{'$date': '2015-08-25T10:01:02.918Z'},"[{'name': 'envelopes', 'tags': ['stationary', ...",Seattle,"{'gender': 'M', 'age': 50, 'email': 'keecade@h...",False,Phone
2,{'$oid': '5bd761dcae323e45a93ccfea'},{'$date': '2017-06-22T09:54:14.185Z'},"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 51, 'email': 'worbiduh@...",False,In store
3,{'$oid': '5bd761dcae323e45a93ccfeb'},{'$date': '2015-02-23T09:53:59.343Z'},"[{'name': 'binder', 'tags': ['school', 'genera...",Seattle,"{'gender': 'F', 'age': 45, 'email': 'vatires@t...",False,In store
4,{'$oid': '5bd761dcae323e45a93ccfec'},{'$date': '2017-12-03T18:39:48.253Z'},"[{'name': 'backpack', 'tags': ['school', 'trav...",London,"{'gender': 'M', 'age': 40, 'email': 'dotzu@ib....",False,In store


### Pre Processing
Extract and transform nested JSON structures which are 'items' and 'customer' fields. We tranform these nested structures into individual rows.

In [12]:
items_df = pd.json_normalize(df['items'].explode()).reset_index(drop=True)
items_df.head()

,name,tags,quantity,price.$numberDecimal
0,printer paper,"[office, stationary]",2,40.01
1,notepad,"[office, writing, school]",2,35.29
2,pens,"[writing, office, school, stationary]",5,56.12
3,backpack,"[school, travel, kids]",2,77.71
4,notepad,"[office, writing, school]",2,18.47


In [13]:
customer_df = pd.json_normalize(df['customer']).reset_index(drop=True)
customer_df.head()

,gender,age,email,satisfaction
0,M,42,cauho@witwuta.sv,4
1,M,50,keecade@hem.uy,5
2,M,51,worbiduh@vowbu.cg,5
3,F,45,vatires@ta.pe,3
4,M,40,dotzu@ib.sh,4


### Task1 - Top 10 products sales
The top 10 products sales based on the multiplcation of quantity and price.

In [14]:
items_df['total_sales'] = items_df['price.$numberDecimal'].astype(float) * items_df['quantity']
top_10_products = items_df[['name', 'total_sales']].sort_values(by='total_sales', ascending=False).head(10)
top_10_products

,name,total_sales
11797,laptop,7966.55
308,laptop,7933.50
2807,laptop,7912.50
21892,laptop,7888.10
16367,laptop,7883.50
24209,laptop,7878.80
7476,laptop,7877.55
27164,laptop,7863.15
13847,laptop,7852.45
7323,laptop,7844.90


In [15]:
items_df['index'] = items_df.index
customer_df['index'] = customer_df.index
main_df = df[['storeLocation', 'purchaseMethod', 'saleDate']].reset_index()
merged_df = pd.merge(items_df, main_df, left_on='index', right_on='index')
merged_df.head()
final_merge_df = pd.merge(merged_df, customer_df, left_on='index', right_on='index')
final_merge_df.head()

,name,tags,quantity,price.$numberDecimal,total_sales,index,storeLocation,purchaseMethod,saleDate,gender,age,email,satisfaction
0,printer paper,"[office, stationary]",2,40.01,80.02,0,Denver,Online,{'$date': '2015-03-23T21:06:49.506Z'},M,42,cauho@witwuta.sv,4
1,notepad,"[office, writing, school]",2,35.29,70.58,1,Seattle,Phone,{'$date': '2015-08-25T10:01:02.918Z'},M,50,keecade@hem.uy,5
2,pens,"[writing, office, school, stationary]",5,56.12,280.60,2,Denver,In store,{'$date': '2017-06-22T09:54:14.185Z'},M,51,worbiduh@vowbu.cg,5
3,backpack,"[school, travel, kids]",2,77.71,155.42,3,Seattle,In store,{'$date': '2015-02-23T09:53:59.343Z'},F,45,vatires@ta.pe,3
4,notepad,"[office, writing, school]",2,18.47,36.94,4,London,In store,{'$date': '2017-12-03T18:39:48.253Z'},M,40,dotzu@ib.sh,4


### Task2 - Top 3 products sales by store
The top 3 products for each of the store. 

In [16]:
grouped_by_store_and_product = final_merge_df.groupby(['storeLocation', 'name']).total_sales.sum().reset_index()

top_3_products_by_store = grouped_by_store_and_product.groupby('storeLocation').apply(lambda x: x.nlargest(3, 'total_sales')).reset_index(drop=True)
top_3_products_by_store

,storeLocation,name,total_sales
0,Austin,laptop,158970.49
1,Austin,backpack,23265.19
2,Austin,binder,13618.05
3,Denver,laptop,409752.19
4,Denver,backpack,44224.19
5,Denver,pens,32667.76
6,London,laptop,201480.89
7,London,backpack,22946.59
8,London,pens,17510.10
9,New York,laptop,177308.46


### Task3 - Store Ranks
Ranking each store based on the sales.

In [17]:
store_sales = merged_df.groupby('storeLocation').total_sales.sum().reset_index()
store_sales['rank'] = store_sales['total_sales'].rank(ascending=False)
store_sales['rank'] = store_sales['rank'].astype(int)
store_sales.sort_values(by='rank', inplace=True)
store_sales

,storeLocation,total_sales,rank
1,Denver,583256.15,1
5,Seattle,370600.53,2
2,London,291162.89,3
0,Austin,237868.30,4
3,New York,233406.01,5
4,San Diego,125525.93,6


### Task4 - Purchase methods by gender

In [18]:
purchase_by_gender = final_merge_df.groupby(['purchaseMethod', 'gender']).size().unstack().reset_index()
purchase_by_gender.columns.name = None 
purchase_by_gender

,purchaseMethod,F,M
0,In store,1430,1389
1,Online,813,772
2,Phone,284,312


### Task5 - Monthly total sales
Displaying monthly total sales for multiple years

In [19]:
final_merge_df['year'] = pd.to_datetime(final_merge_df['saleDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)).dt.year
final_merge_df['month'] = pd.to_datetime(final_merge_df['saleDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)).dt.month
monthly_sales = final_merge_df.groupby(['year', 'month']).total_sales.sum().reset_index().sort_values(by=['year', 'month'])
monthly_sales.head(24)

,year,month,total_sales
0,2013,1,22516.78
1,2013,2,29306.27
2,2013,3,19943.84
3,2013,4,38496.15
4,2013,5,27018.06
5,2013,6,33785.74
6,2013,7,25126.67
7,2013,8,20167.53
8,2013,9,27709.83
9,2013,10,29965.56
